In [1]:
import os
import sys
import time as t_util
import numpy as np
import pandas as pd
import yaml
import cftime
import xarray as xr


In [2]:
#Read main paths
with open('../path_main.txt', 'r') as file:   path_main = file.read()
with open('../path_CMIP6.txt', 'r') as file:  path_cmip6 = file.read()
    
dir_CMIP6   = path_cmip6 
dir_scripts = f'{path_main}Scripts/'
dir_names   = f'{path_main}Scripts/Model_lists/'
dir_Yearmax = f'{path_main}Data/CMIP6/Yearmax/'
if not os.path.exists(dir_Yearmax):  os.mkdir(dir_Yearmax)


## Prepare variables and parameters

In [8]:
#Define models and SSPs which should be used
all_models = dict()
all_models['ssp585'] = []
with open(dir_names + 'Models_CMIP6_SSP585.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['ssp585'].append(line[:-1])

#Define time periods
time_periods = [[1981, 2010], [2070, 2099]]

scenarios = ['historical', 'ssp585']

variables = ['tasmax', 'tasmin']
vars_out = dict()
vars_out['tasmax'] = 'TXx'
vars_out['tasmin'] = 'TNx'


## Calculation

In [9]:
#Loop over models
for model in all_models['ssp585']:

    print(model, end=': ')
    
    #Select ensemble member
    if model in ['CNRM-CM6-1', 'CNRM-ESM2-1', 'CNRM-CM6-1-HR', 'UKESM1-0-LL', 'MIROC-ES2L']:
        member = "r1i1p1f2"
    elif model in ['HadGEM3-GC31-LL', 'HadGEM3-GC31-MM']:
        member = "r1i1p1f3"
    else:
        member = "r1i1p1f1"            

    #Loop over variables
    for variab in variables:

        print(variab, end=', ')
        
        #Creat output directory
        dir_Ymax_out = dir_Yearmax + vars_out[variab] + '/'
        if not os.path.exists(dir_Ymax_out):  os.mkdir(dir_Ymax_out)          
        
        #Get file names
        fnames_merge = []
        for scen in scenarios:
        
            dir_data = dir_CMIP6 + scen + '/' + variab + '/'
            fnames = [dir_data + file for file in os.listdir(dir_data) if model + '_' in file and member in file and variab in file and scen in file and '_day_' in file]
            if len(fnames)!=1: sys.exit('Filename not uniquely defined')

            #Add to list for all files for merging
            fnames_merge = fnames_merge + fnames

        #Read and merge data
        data = xr.concat((xr.open_dataset(fname) for fname in fnames_merge), dim='time')
        data = data.rename({variab: vars_out[variab]})
        
        #Loop over time periods
        for time_per in time_periods:

            #Select time
            time_sel = slice(str(time_per[0]), str(time_per[1]))
            data_sel = data.sel(time=time_sel)

            #Calculate yearly maximum
            data_Yearmax  = data_sel.resample(time='1Y').max()

            #Create output file names
            output_str = "_" + model + '_' + "-".join(scenarios) +  "_" + time_sel.start + "-" + time_sel.stop + ".nc"
            fname_Yearmax = dir_Ymax_out + vars_out[variab] + "_Yearmax" + output_str

            #Save in file
            if os.path.exists(fname_Yearmax): os.remove(fname_Yearmax)
            data_Yearmax.to_netcdf(fname_Yearmax)


KACE-1-0-G: tasmin, 